In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


"""
Selva - notes
onehot_encode = OneHotEncoder()
print (onehot_encode)

#one hot encoding the categorical values
feature_array = onehot_encode.fit_transform(X[["Pclass","Sex","SibSp","Parch","Cabin","Embarked"]]).toarray()

#labelling the columns with the categorical features
feature_labels = onehot_encode.categories_
#now we have to flaten the list since feature_labels has nested list in it
flattened_label = []
for sublist in feature_labels:
    for items in sublist:
        flattened_label.append(items)

print (flattened_label)
#np.array(feature_labels).ravel()
"""

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


'\nSelva - notes\nonehot_encode = OneHotEncoder()\nprint (onehot_encode)\n\n#one hot encoding the categorical values\nfeature_array = onehot_encode.fit_transform(X[["Pclass","Sex","SibSp","Parch","Cabin","Embarked"]]).toarray()\n\n#labelling the columns with the categorical features\nfeature_labels = onehot_encode.categories_\n#now we have to flaten the list since feature_labels has nested list in it\nflattened_label = []\nfor sublist in feature_labels:\n    for items in sublist:\n        flattened_label.append(items)\n\nprint (flattened_label)\n#np.array(feature_labels).ravel()\n'

### Understanding the Data

In [2]:
import pandas as pd
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
print (train_data.columns)

#Embarked has only 2 NaN values so removing the rows now
train_data = train_data.dropna(subset = ["Embarked"])
#Also dropping Text features since they are not useful in this classification
#X = X.drop(columns = ["PassengerId", "Name", "Ticket"])

#splitting the training data into Y (predicted variable) and X 
#(Independent variable - predictor of X)
X = train_data.drop(['Survived'], axis = 1)
Y = train_data['Survived']

print (X.shape)

#print ("Independent variables(X): ", X[:4])
#print ("To be predicted variables(Y): ", Y[:4])

# Check for NaN values in each column
nan_columns = X.columns[X.isna().any()].tolist()
print (X['Age'].isna().sum())
print (X['Cabin'].isna().sum())
print (X['Embarked'].isna().sum())
print(nan_columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
(889, 11)
177
687
0
['Age', 'Cabin']


### Feature Selection

-- <font color = 'green'>Finding relevant features in the dataset X which will influence the Y value </font>

In [3]:
#the dtypes gives the data types of each variable
print (X.dtypes)
# from the above we get name,sex, ticket, cabin and embarked as object data types.
#It is clear that name does not need to be one hot encoded as it doesn't result in any
#useful prediction. 
#To confirm embarked, cabin and sex lets perform a group by search to confirm if they 
#can be one hot encoded.

# doing a distinct search in categorical features for one hot encoding.

cabin_count = pd.unique(X['Cabin'])
print ("Cabin, ", X["Cabin"].unique())
#print (len(cabin_count))
#print (len(X))
print ("Sex: ", X["Sex"].unique())
print ("Pclass: ", X["Pclass"].unique()) #categorical feature one hot encode pending
print("SibSp: ", X["SibSp"].unique()) #categorical feature one hot encode pending
print ("Parch: ", X["Parch"].unique())


print(X["Embarked"].isna().sum())
print (X["Cabin"].isna().sum())
print (X["Age"].isna().sum())

print(X.shape)

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object
Cabin,  [nan 'C85' 'C123' 'E46' 'G6' 'C103' 'D56' 'A6' 'C23 C25 C27' 'B78' 'D33'
 'B30' 'C52' 'C83' 'F33' 'F G73' 'E31' 'A5' 'D10 D12' 'D26' 'C110'
 'B58 B60' 'E101' 'F E69' 'D47' 'B86' 'F2' 'C2' 'E33' 'B19' 'A7' 'C49'
 'F4' 'A32' 'B4' 'B80' 'A31' 'D36' 'D15' 'C93' 'C78' 'D35' 'C87' 'B77'
 'E67' 'B94' 'C125' 'C99' 'C118' 'D7' 'A19' 'B49' 'D' 'C22 C26' 'C106'
 'C65' 'E36' 'C54' 'B57 B59 B63 B66' 'C7' 'E34' 'C32' 'B18' 'C124' 'C91'
 'E40' 'T' 'C128' 'D37' 'B35' 'E50' 'C82' 'B96 B98' 'E10' 'E44' 'A34'
 'C104' 'C111' 'C92' 'E38' 'D21' 'E12' 'E63' 'A14' 'B37' 'C30' 'D20' 'B79'
 'E25' 'D46' 'B73' 'C95' 'B38' 'B39' 'B22' 'C86' 'C70' 'A16' 'C101' 'C68'
 'A10' 'E68' 'B41' 'A20' 'D19' 'D50' 'D9' 'A23' 'B50' 'A26' 'D48' 'E58'
 'C126' 

### One hot encoding

--<font color = 'green'> based on the above code the features Cabin, Sex and Embarked are categorical features, so lets convert them to numerical data with one hot encoding. </font>

In [4]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
import numpy as np

def data_preprocessing(X):
    #Drop 
    X = X.drop(columns=["PassengerId", "Name", "Ticket"])
    #nan_columns_infun = X.columns[X.isna().any()].tolist()
    #print ("Before removing NAN columns: ",nan_columns_infun)
    
    ## dealing with the Nan values
    X["Cabin"] = X["Cabin"].fillna("NA")
    age_mean = X["Age"].mean()
    X["Age"] = X["Age"].fillna(age_mean)
    
    ## Step 2: Scale the numerical features to lie between 0 and 1
    # I am using MinMax scaler.
    scaler = MinMaxScaler()
    scaled_features = scaler.fit_transform(X[["Age", "Fare"]])
    scaled_df = pd.DataFrame(scaled_features, columns=["Age", "Fare"])
    X = X.drop(columns=["Age", "Fare"])
    X = pd.concat([X.reset_index(drop=True), scaled_df.reset_index(drop=True)], axis=1)
    #X = pd.concat([X,scaled_df], axis=1)
    
    #nan_columns_infun = X.columns[X.isna().any()].tolist()
    #print ("NAN columns: ",nan_columns_infun)
    
    ##Step 1: One hot encoding categorical features
    onehot_encode = OneHotEncoder() #print (onehot_encode)
    
    feature_array = onehot_encode.fit_transform(X[["Pclass","Sex","SibSp","Parch","Cabin","Embarked"]]).toarray()
    feature_labels = onehot_encode.categories_ 
    print(feature_labels)
    
    #dropping the encoded features
    X = X.drop(columns = ["Pclass","Sex","SibSp","Parch","Cabin","Embarked"])
    
    flattened_label = []
    for sublist in feature_labels:
        for items in sublist:
            flattened_label.append(items)
    
    #print (flattened_label)
    features = pd.DataFrame(feature_array, columns = flattened_label)
    X = pd.concat([X,features], axis = 1)
    nan_columns_infun = X.columns[X.isna().any()].tolist()
    print (nan_columns_infun)
    print (X.shape)
    return X



In [5]:
X_new = data_preprocessing(X)

[array([1, 2, 3]), array(['female', 'male'], dtype=object), array([0, 1, 2, 3, 4, 5, 8]), array([0, 1, 2, 3, 4, 5, 6]), array(['A10', 'A14', 'A16', 'A19', 'A20', 'A23', 'A24', 'A26', 'A31',
       'A32', 'A34', 'A36', 'A5', 'A6', 'A7', 'B101', 'B102', 'B18',
       'B19', 'B20', 'B22', 'B3', 'B30', 'B35', 'B37', 'B38', 'B39', 'B4',
       'B41', 'B42', 'B49', 'B5', 'B50', 'B51 B53 B55', 'B57 B59 B63 B66',
       'B58 B60', 'B69', 'B71', 'B73', 'B77', 'B78', 'B79', 'B80',
       'B82 B84', 'B86', 'B94', 'B96 B98', 'C101', 'C103', 'C104', 'C106',
       'C110', 'C111', 'C118', 'C123', 'C124', 'C125', 'C126', 'C128',
       'C148', 'C2', 'C22 C26', 'C23 C25 C27', 'C30', 'C32', 'C45', 'C46',
       'C47', 'C49', 'C50', 'C52', 'C54', 'C62 C64', 'C65', 'C68', 'C7',
       'C70', 'C78', 'C82', 'C83', 'C85', 'C86', 'C87', 'C90', 'C91',
       'C92', 'C93', 'C95', 'C99', 'D', 'D10 D12', 'D11', 'D15', 'D17',
       'D19', 'D20', 'D21', 'D26', 'D28', 'D30', 'D33', 'D35', 'D36',
       'D37', 'D45

In [6]:

#print(type(X_new))
#print (type(Y))

print (X_new)
print (X_new.columns)

nan_columns = X.columns[X.isna().any()].tolist()
print (nan_columns)

          Age      Fare    1    2    3  female  male    0    1    2  ...   F2  \
0    0.271174  0.014151  0.0  0.0  1.0     0.0   1.0  0.0  1.0  0.0  ...  0.0   
1    0.472229  0.139136  1.0  0.0  0.0     1.0   0.0  0.0  1.0  0.0  ...  0.0   
2    0.321438  0.015469  0.0  0.0  1.0     1.0   0.0  1.0  0.0  0.0  ...  0.0   
3    0.434531  0.103644  1.0  0.0  0.0     1.0   0.0  0.0  1.0  0.0  ...  0.0   
4    0.434531  0.015713  0.0  0.0  1.0     0.0   1.0  1.0  0.0  0.0  ...  0.0   
..        ...       ...  ...  ...  ...     ...   ...  ...  ...  ...  ...  ...   
884  0.334004  0.025374  0.0  1.0  0.0     0.0   1.0  1.0  0.0  0.0  ...  0.0   
885  0.233476  0.058556  1.0  0.0  0.0     1.0   0.0  1.0  0.0  0.0  ...  0.0   
886  0.367204  0.045771  0.0  0.0  1.0     1.0   0.0  0.0  1.0  0.0  ...  0.0   
887  0.321438  0.058556  1.0  0.0  0.0     0.0   1.0  1.0  0.0  0.0  ...  0.0   
888  0.396833  0.015127  0.0  0.0  1.0     0.0   1.0  1.0  0.0  0.0  ...  0.0   

     F33  F38   F4   G6   N

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_new, Y, test_size = 0.2)

print ('Training dataset size: ',X_train.shape)
print ('Test dataset size: ', X_test.shape)
print ('Training target size: ', Y_train.shape)
print ('Test target size: ', Y_test.shape)


Training dataset size:  (711, 171)
Test dataset size:  (178, 171)
Training target size:  (711,)
Test target size:  (178,)


In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

param_grid_randomForest = {
    'n_estimators' : [100, 200, 300, 400],
    'criterion' : ['gini', 'entropy'],
    'max_depth' : [3, 5, 7, 9, 11, 13]
}
model = RandomForestClassifier()

#initialize gridSearchCV
grid_search = GridSearchCV(estimator = model, param_grid = param_grid_randomForest, cv = 5)
X_train.columns = X_train.columns.astype(str)
#Fit the model to train data
grid_search.fit (X_train, Y_train)
#Get the best model 
best_model = grid_search.best_estimator_
X_test.columns = X_test.columns.astype(str)
y_pred = best_model.predict(X_test)

#calculate the accuracy
accuracy = accuracy_score (Y_test, y_pred)
print(accuracy)

0.7921348314606742
